## Naieve Bayes Model for Sentiment Analysis Predictions

In [3]:
#reset
%rm -rf stonks/

In [4]:
# import required packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB # The Naive Bayes algo

In [5]:
## Retrieving Master CSV data from github for all News Outlet Averages
%%capture
!git clone https://github.com/IS737StockPicker/stonks.git

In [6]:
# Loading the data
master_data_CSV = pd.read_csv('/content/stonks/master_data.csv')
master_data_CSV

,Date,XLE,XLF,XLU,XLI,XLK,XLV,XLY,IYR,AAPL,...,Huff_headline_min,Huff_body_min,NYT_headline_mean,NYT_headline_median,NYT_body_mean,NYT_body_median,NYT_headline_max,NYT_headline_min,NYT_body_max,NYT_body_min
0,2022-03-14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.80,-0.67,-0.05,0.0,-0.03,0.00,0.88,-0.73,0.91,-0.96
1,2022-03-15,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,...,-0.76,-0.85,-0.09,0.0,0.02,0.00,0.84,-0.86,0.88,-0.91
2,2022-03-16,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,-0.25,-0.56,-0.06,0.0,-0.00,0.00,0.86,-0.80,0.88,-0.90
3,2022-03-17,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,-0.90,-0.49,-0.02,0.0,-0.05,0.00,0.61,-0.84,0.82,-0.89
4,2022-03-18,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,-0.60,-0.76,-0.06,0.0,-0.05,-0.08,0.83,-0.82,0.86,-0.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,2022-09-07,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,-0.78,-0.57,0.09,0.0,-0.07,0.00,0.65,-0.86,0.84,-0.93
121,2022-09-08,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,-0.77,-0.83,-0.04,0.0,-0.02,0.00,0.75,-0.90,0.92,-0.93
122,2022-09-09,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,-0.60,0.00,-0.01,0.0,0.14,0.14,0.81,-0.80,0.85,-0.91
123,2022-09-12,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,-0.79,0.00,-0.10,0.0,-0.06,-0.13,0.75,-0.90,0.89,-0.89


In [7]:
#Seperating Tickers from CSV Dataset
tickers = master_data_CSV.iloc[:,:15]
tickers = tickers.drop(columns=['Date'])
tickers

,XLE,XLF,XLU,XLI,XLK,XLV,XLY,IYR,AAPL,XLB,XLP,SPY,^DJI,NDX
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
121,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
122,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
123,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0


In [8]:
#Gettting the Average across News Outlets for each measure (Avg, median, min, max) to get model accuracy for each ticker


#1 Headline Median Avg
accuracy_scores={}
for ticker in tickers.columns:
  selected_var = [ticker,'guardian_headline_median','Huff_headline_median','NYT_headline_median']
  accuracy_df = master_data_CSV[selected_var]
  y = accuracy_df[ticker]
  X = accuracy_df.drop(columns=[ticker])
  X_train = X[:90]
  y_train = y[:90]
  X_valid = X[35:]
  y_valid = y[35:]
  stock_nnet = BernoulliNB(force_alpha=True)
  stock_nnet.fit(X_train, y_train.ravel())
  y_pred = stock_nnet.predict(X_valid)
  accuracy_scores[ticker]= accuracy_score(y_valid, y_pred)
accuracy_scores

{'XLE': 0.4888888888888889,
 'XLF': 0.4888888888888889,
 'XLU': 0.45555555555555555,
 'XLI': 0.6111111111111112,
 'XLK': 0.5555555555555556,
 'XLV': 0.5888888888888889,
 'XLY': 0.4777777777777778,
 'IYR': 0.4666666666666667,
 'AAPL': 0.5555555555555556,
 'XLB': 0.5222222222222223,
 'XLP': 0.5333333333333333,
 'SPY': 0.5777777777777777,
 '^DJI': 0.5222222222222223,
 'NDX': 0.45555555555555555}

In [9]:
#2 Headline Mean Avg
accuracy_scores={}
for ticker in tickers.columns:
  selected_var = [ticker,'guardian_headline_mean','Huff_headline_mean','NYT_headline_mean']
  accuracy_df = master_data_CSV[selected_var]
  y = accuracy_df[ticker]
  X = accuracy_df.drop(columns=[ticker])
  X_train = X[:90]
  y_train = y[:90]
  X_valid = X[35:]
  y_valid = y[35:]
  stock_nnet = BernoulliNB(force_alpha=True)
  stock_nnet.fit(X_train, y_train.ravel())
  y_pred = stock_nnet.predict(X_valid)
  accuracy_scores[ticker]= accuracy_score(y_valid, y_pred)
accuracy_scores

{'XLE': 0.5,
 'XLF': 0.5444444444444444,
 'XLU': 0.4888888888888889,
 'XLI': 0.6111111111111112,
 'XLK': 0.5333333333333333,
 'XLV': 0.5888888888888889,
 'XLY': 0.5222222222222223,
 'IYR': 0.5,
 'AAPL': 0.5555555555555556,
 'XLB': 0.5333333333333333,
 'XLP': 0.5333333333333333,
 'SPY': 0.5333333333333333,
 '^DJI': 0.5222222222222223,
 'NDX': 0.43333333333333335}

In [10]:
#3 Body Median Avg
accuracy_scores={}
for ticker in tickers.columns:
  selected_var = [ticker,'guardian_body_median','Huff_body_median','NYT_body_median']
  accuracy_df = master_data_CSV[selected_var]
  y = accuracy_df[ticker]
  X = accuracy_df.drop(columns=[ticker])
  X_train = X[:90]
  y_train = y[:90]
  X_valid = X[35:]
  y_valid = y[35:]
  stock_nnet = BernoulliNB(force_alpha=True)
  stock_nnet.fit(X_train, y_train.ravel())
  y_pred = stock_nnet.predict(X_valid)
  accuracy_scores[ticker]= accuracy_score(y_valid, y_pred)
accuracy_scores

{'XLE': 0.5333333333333333,
 'XLF': 0.5,
 'XLU': 0.5222222222222223,
 'XLI': 0.5444444444444444,
 'XLK': 0.5888888888888889,
 'XLV': 0.5444444444444444,
 'XLY': 0.5777777777777777,
 'IYR': 0.5222222222222223,
 'AAPL': 0.5777777777777777,
 'XLB': 0.5111111111111111,
 'XLP': 0.5333333333333333,
 'SPY': 0.5555555555555556,
 '^DJI': 0.5,
 'NDX': 0.5555555555555556}

In [11]:
#4 Body Mean Avg
accuracy_scores={}
for ticker in tickers.columns:
  selected_var = [ticker,'guardian_body_mean','Huff_body_mean','NYT_body_mean']
  accuracy_df = master_data_CSV[selected_var]
  y = accuracy_df[ticker]
  X = accuracy_df.drop(columns=[ticker])
  X_train = X[:90]
  y_train = y[:90]
  X_valid = X[35:]
  y_valid = y[35:]
  stock_nnet = BernoulliNB(force_alpha=True)
  stock_nnet.fit(X_train, y_train.ravel())
  y_pred = stock_nnet.predict(X_valid)
  accuracy_scores[ticker]= accuracy_score(y_valid, y_pred)
accuracy_scores

{'XLE': 0.5555555555555556,
 'XLF': 0.4888888888888889,
 'XLU': 0.5444444444444444,
 'XLI': 0.5666666666666667,
 'XLK': 0.5666666666666667,
 'XLV': 0.6555555555555556,
 'XLY': 0.5333333333333333,
 'IYR': 0.5,
 'AAPL': 0.5333333333333333,
 'XLB': 0.5333333333333333,
 'XLP': 0.5444444444444444,
 'SPY': 0.5333333333333333,
 '^DJI': 0.4888888888888889,
 'NDX': 0.5333333333333333}

In [12]:
#5 Min/Mix headline avg

accuracy_scores={}
for ticker in tickers.columns:
  selected_var = [ticker,'guardian_headline_min','guardian_headline_max','Huff_headline_min','Huff_headline_max','NYT_headline_min','NYT_headline_max']
  accuracy_df = master_data_CSV[selected_var]
  y = accuracy_df[ticker]
  X = accuracy_df.drop(columns=[ticker])
  X_train = X[:90]
  y_train = y[:90]
  X_valid = X[35:]
  y_valid = y[35:]
  stock_nnet = BernoulliNB(force_alpha=True)
  stock_nnet.fit(X_train, y_train.ravel())
  y_pred = stock_nnet.predict(X_valid)
  accuracy_scores[ticker]= accuracy_score(y_valid, y_pred)
accuracy_scores

{'XLE': 0.5777777777777777,
 'XLF': 0.5777777777777777,
 'XLU': 0.5333333333333333,
 'XLI': 0.6222222222222222,
 'XLK': 0.5888888888888889,
 'XLV': 0.5666666666666667,
 'XLY': 0.5444444444444444,
 'IYR': 0.45555555555555555,
 'AAPL': 0.5888888888888889,
 'XLB': 0.5444444444444444,
 'XLP': 0.5111111111111111,
 'SPY': 0.6111111111111112,
 '^DJI': 0.5333333333333333,
 'NDX': 0.5}

In [13]:
#6 Min/Mix body avg

accuracy_scores={}
for ticker in tickers.columns:
  selected_var = [ticker,'guardian_body_min','guardian_body_max','Huff_body_min','Huff_body_max','NYT_body_min','NYT_body_max']
  accuracy_df = master_data_CSV[selected_var]
  y = accuracy_df[ticker]
  X = accuracy_df.drop(columns=[ticker])
  X_train = X[:90]
  y_train = y[:90]
  X_valid = X[35:]
  y_valid = y[35:]
  stock_nnet = BernoulliNB(force_alpha=True)
  stock_nnet.fit(X_train, y_train.ravel())
  y_pred = stock_nnet.predict(X_valid)
  accuracy_scores[ticker]= accuracy_score(y_valid, y_pred)
accuracy_scores

{'XLE': 0.5666666666666667,
 'XLF': 0.4444444444444444,
 'XLU': 0.4666666666666667,
 'XLI': 0.6111111111111112,
 'XLK': 0.6,
 'XLV': 0.5888888888888889,
 'XLY': 0.5111111111111111,
 'IYR': 0.5111111111111111,
 'AAPL': 0.6,
 'XLB': 0.5444444444444444,
 'XLP': 0.5333333333333333,
 'SPY': 0.5777777777777777,
 '^DJI': 0.5444444444444444,
 'NDX': 0.5777777777777777}

## Using Naive Bayes Model to predict future stock prices

In [14]:
# Loading future data set
master_data_future = pd.read_csv('/content/stonks/master_data_future.csv')
master_data_future

,Date,XLE,XLF,XLU,XLI,XLK,XLV,XLY,IYR,AAPL,...,Huff_headline_min,Huff_body_min,NYT_headline_mean,NYT_headline_median,NYT_body_mean,NYT_body_median,NYT_headline_max,NYT_headline_min,NYT_body_max,NYT_body_min
0,2022-03-14,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,...,-0.80,-0.67,-0.05,0.0,-0.03,0.00,0.88,-0.73,0.91,-0.96
1,2022-03-15,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,-0.76,-0.85,-0.09,0.0,0.02,0.00,0.84,-0.86,0.88,-0.91
2,2022-03-16,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,-0.25,-0.56,-0.06,0.0,-0.00,0.00,0.86,-0.80,0.88,-0.90
3,2022-03-17,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,-0.90,-0.49,-0.02,0.0,-0.05,0.00,0.61,-0.84,0.82,-0.89
4,2022-03-18,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,...,-0.60,-0.76,-0.06,0.0,-0.05,-0.08,0.83,-0.82,0.86,-0.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,2022-09-07,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,-0.78,-0.57,0.09,0.0,-0.07,0.00,0.65,-0.86,0.84,-0.93
121,2022-09-08,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,-0.77,-0.83,-0.04,0.0,-0.02,0.00,0.75,-0.90,0.92,-0.93
122,2022-09-09,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,-0.60,0.00,-0.01,0.0,0.14,0.14,0.81,-0.80,0.85,-0.91
123,2022-09-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.79,0.00,-0.10,0.0,-0.06,-0.13,0.75,-0.90,0.89,-0.89


In [15]:
#Seperating Tickers from CSV Dataset
future_tickers = master_data_future.iloc[:,:15]
future_tickers = future_tickers.drop(columns=['Date'])
future_tickers

,XLE,XLF,XLU,XLI,XLK,XLV,XLY,IYR,AAPL,XLB,XLP,SPY,^DJI,NDX
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
1,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
121,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
122,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0
123,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
#Gettting the Average across News Outlets for each measure (Avg, median, min, max) to get model accuracy for each ticker


#7 Future Headline Median Avg
accuracy_scores={}
for ticker in tickers.columns:
  selected_var = [ticker,'guardian_headline_median','Huff_headline_median','NYT_headline_median']
  accuracy_df = master_data_future[selected_var]
  y = accuracy_df[ticker]
  X = accuracy_df.drop(columns=[ticker])
  X_train = X[:90]
  y_train = y[:90]
  X_valid = X[35:]
  y_valid = y[35:]
  stock_nnet = BernoulliNB(force_alpha=True)
  stock_nnet.fit(X_train, y_train.ravel())
  y_pred = stock_nnet.predict(X_valid)
  accuracy_scores[ticker]= accuracy_score(y_valid, y_pred)
accuracy_scores

{'XLE': 0.5777777777777777,
 'XLF': 0.4777777777777778,
 'XLU': 0.5666666666666667,
 'XLI': 0.6,
 'XLK': 0.5444444444444444,
 'XLV': 0.5777777777777777,
 'XLY': 0.4777777777777778,
 'IYR': 0.4777777777777778,
 'AAPL': 0.5555555555555556,
 'XLB': 0.5444444444444444,
 'XLP': 0.5333333333333333,
 'SPY': 0.5777777777777777,
 '^DJI': 0.5111111111111111,
 'NDX': 0.4666666666666667}

In [18]:
#8 Future Headline Mean Avg
accuracy_scores={}
for ticker in tickers.columns:
  selected_var = [ticker,'guardian_headline_mean','Huff_headline_mean','NYT_headline_mean']
  accuracy_df = master_data_future[selected_var]
  y = accuracy_df[ticker]
  X = accuracy_df.drop(columns=[ticker])
  X_train = X[:90]
  y_train = y[:90]
  X_valid = X[35:]
  y_valid = y[35:]
  stock_nnet = BernoulliNB(force_alpha=True)
  stock_nnet.fit(X_train, y_train.ravel())
  y_pred = stock_nnet.predict(X_valid)
  accuracy_scores[ticker]= accuracy_score(y_valid, y_pred)
accuracy_scores

{'XLE': 0.5555555555555556,
 'XLF': 0.5555555555555556,
 'XLU': 0.5444444444444444,
 'XLI': 0.5333333333333333,
 'XLK': 0.5666666666666667,
 'XLV': 0.5777777777777777,
 'XLY': 0.5555555555555556,
 'IYR': 0.4888888888888889,
 'AAPL': 0.6222222222222222,
 'XLB': 0.5666666666666667,
 'XLP': 0.5111111111111111,
 'SPY': 0.5555555555555556,
 '^DJI': 0.5333333333333333,
 'NDX': 0.5888888888888889}

In [19]:
#9 Future Body Median Avg
accuracy_scores={}
for ticker in tickers.columns:
  selected_var = [ticker,'guardian_body_median','Huff_body_median','NYT_body_median']
  accuracy_df = master_data_future[selected_var]
  y = accuracy_df[ticker]
  X = accuracy_df.drop(columns=[ticker])
  X_train = X[:90]
  y_train = y[:90]
  X_valid = X[35:]
  y_valid = y[35:]
  stock_nnet = BernoulliNB(force_alpha=True)
  stock_nnet.fit(X_train, y_train.ravel())
  y_pred = stock_nnet.predict(X_valid)
  accuracy_scores[ticker]= accuracy_score(y_valid, y_pred)
accuracy_scores

{'XLE': 0.5888888888888889,
 'XLF': 0.5,
 'XLU': 0.6,
 'XLI': 0.5444444444444444,
 'XLK': 0.5222222222222223,
 'XLV': 0.5777777777777777,
 'XLY': 0.4888888888888889,
 'IYR': 0.5777777777777777,
 'AAPL': 0.5,
 'XLB': 0.5555555555555556,
 'XLP': 0.5333333333333333,
 'SPY': 0.5555555555555556,
 '^DJI': 0.5111111111111111,
 'NDX': 0.43333333333333335}

In [20]:
#10 Future Body Mean Avg
accuracy_scores={}
for ticker in tickers.columns:
  selected_var = [ticker,'guardian_body_mean','Huff_body_mean','NYT_body_mean']
  accuracy_df = master_data_future[selected_var]
  y = accuracy_df[ticker]
  X = accuracy_df.drop(columns=[ticker])
  X_train = X[:90]
  y_train = y[:90]
  X_valid = X[35:]
  y_valid = y[35:]
  stock_nnet = BernoulliNB(force_alpha=True)
  stock_nnet.fit(X_train, y_train.ravel())
  y_pred = stock_nnet.predict(X_valid)
  accuracy_scores[ticker]= accuracy_score(y_valid, y_pred)
accuracy_scores

{'XLE': 0.5888888888888889,
 'XLF': 0.5333333333333333,
 'XLU': 0.5222222222222223,
 'XLI': 0.5777777777777777,
 'XLK': 0.5333333333333333,
 'XLV': 0.5777777777777777,
 'XLY': 0.4222222222222222,
 'IYR': 0.5666666666666667,
 'AAPL': 0.5333333333333333,
 'XLB': 0.4666666666666667,
 'XLP': 0.5111111111111111,
 'SPY': 0.5555555555555556,
 '^DJI': 0.5222222222222223,
 'NDX': 0.5111111111111111}

In [21]:
 #11 Future Min/Mix headline avg

accuracy_scores={}
for ticker in tickers.columns:
  selected_var = [ticker,'guardian_headline_min','guardian_headline_max','Huff_headline_min','Huff_headline_max','NYT_headline_min','NYT_headline_max']
  accuracy_df = master_data_future[selected_var]
  y = accuracy_df[ticker]
  X = accuracy_df.drop(columns=[ticker])
  X_train = X[:90]
  y_train = y[:90]
  X_valid = X[35:]
  y_valid = y[35:]
  stock_nnet = BernoulliNB(force_alpha=True)
  stock_nnet.fit(X_train, y_train.ravel())
  y_pred = stock_nnet.predict(X_valid)
  accuracy_scores[ticker]= accuracy_score(y_valid, y_pred)
accuracy_scores

{'XLE': 0.5777777777777777,
 'XLF': 0.5333333333333333,
 'XLU': 0.5,
 'XLI': 0.6111111111111112,
 'XLK': 0.5777777777777777,
 'XLV': 0.5777777777777777,
 'XLY': 0.5222222222222223,
 'IYR': 0.5333333333333333,
 'AAPL': 0.5888888888888889,
 'XLB': 0.5111111111111111,
 'XLP': 0.4666666666666667,
 'SPY': 0.5888888888888889,
 '^DJI': 0.5444444444444444,
 'NDX': 0.5111111111111111}

In [22]:
#12 Future Min/Mix body avg

accuracy_scores={}
for ticker in tickers.columns:
  selected_var = [ticker,'guardian_body_min','guardian_body_max','Huff_body_min','Huff_body_max','NYT_body_min','NYT_body_max']
  accuracy_df = master_data_future[selected_var]
  y = accuracy_df[ticker]
  X = accuracy_df.drop(columns=[ticker])
  X_train = X[:90]
  y_train = y[:90]
  X_valid = X[35:]
  y_valid = y[35:]
  stock_nnet = BernoulliNB(force_alpha=True)
  stock_nnet.fit(X_train, y_train.ravel())
  y_pred = stock_nnet.predict(X_valid)
  accuracy_scores[ticker]= accuracy_score(y_valid, y_pred)
accuracy_scores

{'XLE': 0.5777777777777777,
 'XLF': 0.5666666666666667,
 'XLU': 0.5444444444444444,
 'XLI': 0.6,
 'XLK': 0.5777777777777777,
 'XLV': 0.5777777777777777,
 'XLY': 0.5666666666666667,
 'IYR': 0.5111111111111111,
 'AAPL': 0.5555555555555556,
 'XLB': 0.6222222222222222,
 'XLP': 0.5333333333333333,
 'SPY': 0.5777777777777777,
 '^DJI': 0.5111111111111111,
 'NDX': 0.5777777777777777}